# Spectroscopic Data Reduction Part 2: Wavelength Calibration

This notebook assumes you've completed the Spectroscopic Trace process (see [Part 1](1-SpectroscopicTraceTutorial.ipynb)) and have a trace model handy.

## Authors
Adam Ginsburg, Kelle Cruz, Lia Corrales, Jonathan Sick, Adrian Price-Whelan

## Learning Goals
* Extract calibration lamp spectra from two-dimensional spectral images
* Fit a wavelength solution


## Keywords
Spectroscopy

## Summary
This tutorial will walk through extraction of a calibration lamp spectrum using an existing trace.

It will then demonstrate line identification using the NIST line list database.

Finally, it will show how to fit a wavelength solution to a calibration spectrum, integrating information from multiple calibration lamps.

In [ ]:
import requests

url = 'https://raw.githubusercontent.com/skgrunblatt/astropy-tutorials/main/tutorials/SpectroscopicDataReductionBasics/requirements.txt'
response = requests.get(url)

if response.status_code == 200:
    print(f"Required packages for this notebook:\n{response.text}")
else:
    print("Failed to retrieve the file.")

In [ ]:
!pip install astroquery

In [ ]:
from astropy import units as u
from astropy.modeling.polynomial import Polynomial1D
from astropy.modeling.models import Gaussian1D, Linear1D
from astropy.modeling.fitting import LinearLSQFitter
from IPython.display import Image

In [ ]:
from PIL import Image as PILImage
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['image.origin'] = 'lower'  # we want to show images, not matrices, so we set the origin to the lower-left
plt.style.use('dark_background')  # Optional configuration: if run, this will look nice on dark background notebooks

Now, we're going to use an emission nebula as our wavelength calibrator. We'll use a spectrum take of NGC 7027 with the 16" telescope.

In [ ]:
from io import BytesIO
# example for a FITS image (uncomment this for final part of lab):
from astropy.io import fits
url = "https://raw.githubusercontent.com/skgrunblatt/astropy-tutorials/main/tutorials/SpectroscopicDataReductionBasics/ngc7027-5mdark.fits"

# Fetch the image data from the URL
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Create a file-like object from the response content
    image_data = fits.open(BytesIO(response.content))[0].data
    print("Image loaded successfully!")
else:
    print("Failed to retrieve the image.")

In [ ]:
plt.imshow(image_data);

We re-create our trace model from the [Spectroscopic Trace Tutorial](Spectroscopic%20Trace%20Tutorial.ipynb) using the fitted models.

(We could have used the empirical trace directly, which might result in slightly improved noise characteristics, but for simplicity - and to make the two notebooks independently usable - we use the fitted polynomial & Gaussian models here)

In [ ]:
trace_model = Polynomial1D(degree=3, c0=53.99807515, c1=-0.01366068, c2=0.00001542, c3=-0.00000002) # plug in coefficients from part 1 with Betelgeuse or Sirius here!
trace_profile_model = Gaussian1D(amplitude=123.84846797, mean=0.17719819, stddev=5.10872134)
xaxis = np.arange(image_data.shape[1])
trace_center = trace_model(xaxis)
npixels_to_cut=15
yaxis = np.arange(-npixels_to_cut, npixels_to_cut)
model_trace_profile = trace_profile_model(yaxis)

In [ ]:
trace_model

In [ ]:
plt.imshow(image_data)
plt.plot(xaxis,trace_center)

Our trace doesn't go through the brightest points of the spectrum, so we'll shift it upwards by several pixels to capture more signal.

In [ ]:
trace_model = Polynomial1D(degree=3, c0=53.99807515+15, c1=-0.01366068, c2=0.00001542, c3=-0.00000002) # plug in coefficients from part 1 with Betelgeuse or Sirius here!
trace_profile_model = Gaussian1D(amplitude=123.84846797, mean=0.17719819, stddev=5.10872134)
xaxis = np.arange(image_data.shape[1])
trace_center = trace_model(xaxis)
npixels_to_cut=15
yaxis = np.arange(-npixels_to_cut, npixels_to_cut)
model_trace_profile = trace_profile_model(yaxis)

In [ ]:
plt.imshow(image_data)
plt.plot(xaxis,trace_center)

Now our trace seems to be passing directly through the emission lines produced by the nebula. Let's look at the resultant spectrum.

In [ ]:
ngc7027_spectrum = np.array([np.average(image_data[int(yval)-npixels_to_cut:int(yval)+npixels_to_cut, ii],
                                weights=model_trace_profile)
                     for yval, ii in zip(trace_center, xaxis)])

In [ ]:
plt.plot(xaxis, ngc7027_spectrum)

The narrow emission lines from the nebula are now clearly prominent over the broader spectral features produced by sources on Earth. We can now compare to reference observations of NGC 7027 at [this website](http://wsdiscovery.free.fr/astronomie/spectro/atlas/np/ngc7027/index.html), and guess the pixel values and the wavelengths of the narrow emission lines seen here.

In [ ]:
guessed_wavelengths = [500.7, 656.3, 496.0]
guessed_xvals = [455, 90, 468]

## Improving on our guesses

We can do a lot better at determining the pixel X-values by taking the intensity-weighted coordinate (moment 1):

In [ ]:
npixels = 5
improved_xval_guesses = [np.average(xaxis[g-npixels:g+npixels],
                                    weights=image_data[0][g-npixels:g+npixels] - np.median(image_data[0]))
                         for g in guessed_xvals]
improved_xval_guesses

In [ ]:
plt.plot(xaxis, ngc7027_spectrum)
plt.plot(guessed_xvals, [30]*len(guessed_wavelengths), 'x')
plt.plot(improved_xval_guesses, [30]*len(guessed_wavelengths), '+');

We only have three data points, but that is enough to fit a linear model and still have a free point to check that we got it close to right:

In [ ]:
linfitter = LinearLSQFitter()

We use a `Linear1D` model because we will want to use its inverse later (other models are not invertible)

In [ ]:
wlmodel = Linear1D()
linfit_wlmodel = linfitter(model=wlmodel, x=improved_xval_guesses, y=guessed_wavelengths)
wavelengths = linfit_wlmodel(xaxis) * u.nm
linfit_wlmodel

Note this fitted slope: each pixel is about 0.1 nm (about 1 angstrom), and the wavelength increases to the left.

In [ ]:
plt.plot(wavelengths, ngc7027_spectrum)
plt.plot(guessed_wavelengths, [100]*len(guessed_wavelengths), 'x');

We show our model $\lambda(x)$ vs the input "guesses":

In [ ]:
plt.plot(improved_xval_guesses, guessed_wavelengths, 'o')
plt.plot(xaxis, wavelengths, '-')
plt.ylabel("$\lambda(x)$")
plt.xlabel("x (pixels)")

Indeed, a linear model fit excellently!